In [3]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

import itertools
import joblib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
columns = [
    # unique id
    u'reservation_id', 
    # dates
    u'booking_date', u'checkin_date', u'checkout_date',
    # traveller related info
    u'memberid', u'numberofadults', u'numberofchildren', u'total_pax', u'persontravellingid', u'member_age_buckets', 
    u'state_code_residence',
    # resort info
    u'resort_id', u'resort_region_code', u'resort_type_code', u'state_code_resort', u'cluster_code',
    # booking info
    u'channel_code', u'booking_type_code', u'room_type_booked_code', u'roomnights', u'season_holidayed_code', 
    u'reservationstatusid_code', 
    # product purchased
    u'main_product_code',
    # target 
    u'amount_spent_per_room_night_scaled'
]


In [5]:
train = pd.read_csv('train.csv')
print train.shape
test = pd.read_csv('test.csv')
print test.shape

(341424, 24)
(146765, 23)


In [6]:
df = pd.concat([train, test], axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df.shape

(488189, 24)

In [7]:
df.booking_date = pd.to_datetime(df.booking_date, format='%d/%m/%y')
df.checkin_date = pd.to_datetime(df.checkin_date, format='%d/%m/%y')
df.checkout_date = pd.to_datetime(df.checkout_date, format='%d/%m/%y')

In [8]:
df = df[columns]
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,memberid,numberofadults,numberofchildren,total_pax,persontravellingid,member_age_buckets,state_code_residence,resort_id,resort_region_code,resort_type_code,state_code_resort,cluster_code,channel_code,booking_type_code,room_type_booked_code,roomnights,season_holidayed_code,reservationstatusid_code,main_product_code,amount_spent_per_room_night_scaled
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,3,46,F,7.0,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,F,3,1,3,1,2.0,C,1,7.706428
1,03930f033646d073462b35d411616323597715ac4fc398...,2015-01-23,2015-04-11,2015-04-16,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,2,46,F,7.0,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,3,3,5,F,1,1,4,5,2.0,A,1,6.662563
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,2015-01-28,2015-02-01,2015-02-05,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,2,47,F,7.0,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,1,5,1,E,1,1,4,4,2.0,A,1,7.871602
3,cfd77f44811ed62f25a220b53324cdbafc662a4c9e5f04...,2015-05-02,2015-06-11,2015-06-16,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,2,2,46,F,7.0,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,2,D,1,1,3,5,2.0,A,1,5.344943
4,937cff9e4dcfc2459620153dfc8b9962ac22bea67dfb29...,2015-09-02,2015-12-14,2015-12-19,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,2,46,F,7.0,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2,2,2,D,1,1,4,5,2.0,A,1,7.059346


In [9]:
df['booking_in_advance_days'] = (df.checkin_date - df.booking_date).dt.days
df['booking_roomnights'] = (df.checkout_date - df.checkin_date).dt.days
df['total_persons_travelling'] = df.numberofadults + df.numberofchildren

In [10]:
# roomnights, numberofadults, numberofchildren, total_pax

In [11]:
def create_date_variables(df, date_key, week=False, month=False, year=False, dayofweek=False, dayofmonth=False, dayofyear=False):
    if week:
        df['{}_{}'.format(date_key, 'week')] = df[date_key].dt.week
    if month:
        df['{}_{}'.format(date_key, 'month')] = df[date_key].dt.month
    if year:
        df['{}_{}'.format(date_key, 'year')] = df[date_key].dt.year
    if dayofweek:
        df['{}_{}'.format(date_key, 'dayofweek')] = df[date_key].dt.dayofweek
    if dayofmonth:
        df['{}_{}'.format(date_key, 'dayofmonth')] = df[date_key].dt.day
    if dayofyear:
        df['{}_{}'.format(date_key, 'dayofyear')] = df[date_key].dt.dayofyear
    return df 

def num_div_interactions(df, num_cols):
    for col1, col2 in itertools.combinations(num_cols,2):
        df["{}_div_{}".format(col1, col2)] = (df[col1]/df[col2]).replace([np.inf, -np.inf, np.nan], 0)
    return df

def num_dif_interactions(df, num_cols):
    for col1, col2 in itertools.combinations(num_cols,2):
        df["{}_dif_{}".format(col1, col2)] = df[col1] - df[col2]
    return df
    
def cat_interactions(df, cat_cols):
    for col1, col2 in itertools.combinations(cat_cols, 2):
        print '{}_{}'.format(col1, col2) 
        df['{}_{}'.format(col1, col2)] = df[col1].astype(str) + '_' + df[col2].astype(str)
    return df

    
def create_group_variables(df, group_col, num_col):
    #     count, mean, std, min, max, x-mean, median, x-median, iqr 
    if isinstance(group_col, str):
        group_col = [group_col]
    if isinstance(num_col, str):
        num_col = [num_col]
        
    df_group = df.groupby(group_col)
    
    functions = {
        'mean':'mean',
        'max':'max',
        'min':'min',
        'std':'std',
        'median':'median',
#         'min-max': lambda x: x.max()- x.min()
    }
    final_columns, apply_dict = [], {}
    prefix = 'GRP_' + '_'.join(map(str, group_col))
    for col in num_col:
        methods = []            
        for key, val in functions.iteritems():
            methods.append(val)
            final_columns.append("{}#{}#{}".format(prefix , col, key)) 
        apply_dict[col] = methods
        
    df_var = df_group.agg(apply_dict)
    df_var.columns = final_columns
    df_var.reset_index(inplace=True)
    
    df = df.merge(df_var, how='left', on=group_col)

    for col in num_col:
        df["{}#{}#{}".format(prefix , col, 'min_mean')] = df[col] - df["{}#{}#{}".format(prefix , col, 'mean')]
        
    return df


In [12]:
# df.head()
df = create_date_variables(df, 'checkin_date',  week=True, month=True, year=True, dayofweek=True, dayofmonth=True, dayofyear=True)
df = create_date_variables(df, 'checkout_date',  week=True, dayofweek=True)
df = create_date_variables(df, 'booking_date',  week=True, month=True, year=True, dayofyear=True)


# df = create_date_variables(df, 'checkout_date')
# df = create_date_variables(df, 'booking_date')

In [13]:
df.columns

Index([u'reservation_id', u'booking_date', u'checkin_date', u'checkout_date',
       u'memberid', u'numberofadults', u'numberofchildren', u'total_pax',
       u'persontravellingid', u'member_age_buckets', u'state_code_residence',
       u'resort_id', u'resort_region_code', u'resort_type_code',
       u'state_code_resort', u'cluster_code', u'channel_code',
       u'booking_type_code', u'room_type_booked_code', u'roomnights',
       u'season_holidayed_code', u'reservationstatusid_code',
       u'main_product_code', u'amount_spent_per_room_night_scaled',
       u'booking_in_advance_days', u'booking_roomnights',
       u'total_persons_travelling', u'checkin_date_week',
       u'checkin_date_month', u'checkin_date_year', u'checkin_date_dayofweek',
       u'checkin_date_dayofmonth', u'checkin_date_dayofyear',
       u'checkout_date_week', u'checkout_date_dayofweek', u'booking_date_week',
       u'booking_date_month', u'booking_date_year', u'booking_date_dayofyear'],
      dtype='object')

In [14]:
df = cat_interactions(df, ['resort_id', 'checkin_date'])
df = cat_interactions(df, ['resort_id', 'checkout_date'])
df = cat_interactions(df, ['resort_id', 'booking_date'])

df = cat_interactions(df, ['resort_id', 'channel_code'])
df = cat_interactions(df, ['resort_id', 'booking_type_code'])
df = cat_interactions(df, ['resort_id', 'reservationstatusid_code'])
df = cat_interactions(df, ['resort_id', 'resort_type_code'])
df = cat_interactions(df, ['resort_id', 'cluster_code'])
df = cat_interactions(df, ['resort_id', 'room_type_booked_code'])


df = cat_interactions(df, ['persontravellingid', 'member_age_buckets', 'state_code_residence', 'state_code_resort'])
df = cat_interactions(df, ['resort_id', 'memberid'])

df = cat_interactions(df, ['memberid', 'channel_code'])
df = cat_interactions(df, ['memberid', 'booking_type_code'])
df = cat_interactions(df, ['memberid', 'reservationstatusid_code'])
df = cat_interactions(df, ['memberid', 'resort_type_code'])
df = cat_interactions(df, ['memberid', 'cluster_code'])
df = cat_interactions(df, ['memberid', 'room_type_booked_code'])

df = cat_interactions(df, ['memberid', 'checkin_date'])
df = cat_interactions(df, ['memberid', 'checkout_date'])
df = cat_interactions(df, ['memberid', 'booking_date'])

df = cat_interactions(df, ['memberid', 'checkin_date_month'])
df = cat_interactions(df, ['memberid', 'checkin_date_week'])
df = cat_interactions(df, ['memberid', 'checkin_date_dayofweek'])
df = cat_interactions(df, ['memberid', 'checkout_date_week'])
df = cat_interactions(df, ['memberid', 'checkout_date_dayofweek'])
df = cat_interactions(df, ['memberid', 'booking_date_week'])


df = cat_interactions(df, ['resort_id', 'booking_roomnights'])
df = cat_interactions(df, ['memberid', 'booking_roomnights'])


resort_id_checkin_date
resort_id_checkout_date
resort_id_booking_date
resort_id_channel_code
resort_id_booking_type_code
resort_id_reservationstatusid_code
resort_id_resort_type_code
resort_id_cluster_code
resort_id_room_type_booked_code
persontravellingid_member_age_buckets
persontravellingid_state_code_residence
persontravellingid_state_code_resort
member_age_buckets_state_code_residence
member_age_buckets_state_code_resort
state_code_residence_state_code_resort
resort_id_memberid
memberid_channel_code
memberid_booking_type_code
memberid_reservationstatusid_code
memberid_resort_type_code
memberid_cluster_code
memberid_room_type_booked_code
memberid_checkin_date
memberid_checkout_date
memberid_booking_date
memberid_checkin_date_month
memberid_checkin_date_week
memberid_checkin_date_dayofweek
memberid_checkout_date_week
memberid_checkout_date_dayofweek
memberid_booking_date_week
resort_id_booking_roomnights
memberid_booking_roomnights


In [15]:
df.columns

Index([u'reservation_id', u'booking_date', u'checkin_date', u'checkout_date',
       u'memberid', u'numberofadults', u'numberofchildren', u'total_pax',
       u'persontravellingid', u'member_age_buckets', u'state_code_residence',
       u'resort_id', u'resort_region_code', u'resort_type_code',
       u'state_code_resort', u'cluster_code', u'channel_code',
       u'booking_type_code', u'room_type_booked_code', u'roomnights',
       u'season_holidayed_code', u'reservationstatusid_code',
       u'main_product_code', u'amount_spent_per_room_night_scaled',
       u'booking_in_advance_days', u'booking_roomnights',
       u'total_persons_travelling', u'checkin_date_week',
       u'checkin_date_month', u'checkin_date_year', u'checkin_date_dayofweek',
       u'checkin_date_dayofmonth', u'checkin_date_dayofyear',
       u'checkout_date_week', u'checkout_date_dayofweek', u'booking_date_week',
       u'booking_date_month', u'booking_date_year', u'booking_date_dayofyear',
       u'resort_id_checkin

In [16]:
df = num_div_interactions(df, num_cols=['numberofadults', 'numberofchildren','total_pax',  
                                        'roomnights', 'booking_in_advance_days', 'booking_roomnights', 
                                        'total_persons_travelling'])

df = num_dif_interactions(df, num_cols=[ 'roomnights', 'booking_roomnights'])
df = num_dif_interactions(df, num_cols=[ 'total_persons_travelling', 'total_pax'])
df = num_dif_interactions(df, num_cols=[ 'checkout_date_week', 'checkin_date_week', 'booking_date_week'])
df.shape
                                        

(488189, 98)

In [17]:
for c in [u'booking_date', u'checkin_date', u'checkout_date',u'memberid', u'resort_id',
        u'resort_id_checkin_date', u'resort_id_checkout_date',
         ]:
    print c
    df = create_group_variables(df, group_col=[c], num_col=[
        u'numberofadults', u'numberofchildren', u'total_pax',u'roomnights', u'booking_in_advance_days'
    ])
    print df.shape

booking_date
(488189, 128)
checkin_date
(488189, 158)
checkout_date
(488189, 188)
memberid
(488189, 218)
resort_id
(488189, 248)
resort_id_checkin_date
(488189, 278)
resort_id_checkout_date
(488189, 308)


In [18]:
df.shape
df.head()

(488189, 308)

,reservation_id,booking_date,checkin_date,checkout_date,memberid,numberofadults,numberofchildren,total_pax,persontravellingid,member_age_buckets,state_code_residence,resort_id,resort_region_code,resort_type_code,state_code_resort,cluster_code,channel_code,booking_type_code,room_type_booked_code,roomnights,season_holidayed_code,reservationstatusid_code,main_product_code,amount_spent_per_room_night_scaled,booking_in_advance_days,booking_roomnights,total_persons_travelling,checkin_date_week,checkin_date_month,checkin_date_year,checkin_date_dayofweek,checkin_date_dayofmonth,checkin_date_dayofyear,checkout_date_week,checkout_date_dayofweek,booking_date_week,booking_date_month,booking_date_year,booking_date_dayofyear,resort_id_checkin_date,resort_id_checkout_date,resort_id_booking_date,resort_id_channel_code,resort_id_booking_type_code,resort_id_reservationstatusid_code,resort_id_resort_type_code,resort_id_cluster_code,resort_id_room_type_booked_code,persontravellingid_member_age_buckets,persontravellingid_state_code_residence,...,GRP_resort_id_checkin_date#total_pax#std,GRP_resort_id_checkin_date#total_pax#max,GRP_resort_id_checkin_date#total_pax#min,GRP_resort_id_checkin_date#total_pax#median,GRP_resort_id_checkin_date#total_pax#mean,GRP_resort_id_checkin_date#roomnights#std,GRP_resort_id_checkin_date#roomnights#max,GRP_resort_id_checkin_date#roomnights#min,GRP_resort_id_checkin_date#roomnights#median,GRP_resort_id_checkin_date#roomnights#mean,GRP_resort_id_checkin_date#booking_in_advance_days#std,GRP_resort_id_checkin_date#booking_in_advance_days#max,GRP_resort_id_checkin_date#booking_in_advance_days#min,GRP_resort_id_checkin_date#booking_in_advance_days#median,GRP_resort_id_checkin_date#booking_in_advance_days#mean,GRP_resort_id_checkin_date#numberofadults#min_mean,GRP_resort_id_checkin_date#numberofchildren#min_mean,GRP_resort_id_checkin_date#total_pax#min_mean,GRP_resort_id_checkin_date#roomnights#min_mean,GRP_resort_id_checkin_date#booking_in_advance_days#min_mean,GRP_resort_id_checkout_date#numberofadults#std,GRP_resort_id_checkout_date#numberofadults#max,GRP_resort_id_checkout_date#numberofadults#min,GRP_resort_id_checkout_date#numberofadults#median,GRP_resort_id_checkout_date#numberofadults#mean,GRP_resort_id_checkout_date#numberofchildren#std,GRP_resort_id_checkout_date#numberofchildren#max,GRP_resort_id_checkout_date#numberofchildren#min,GRP_resort_id_checkout_date#numberofchildren#median,GRP_resort_id_checkout_date#numberofchildren#mean,GRP_resort_id_checkout_date#total_pax#std,GRP_resort_id_checkout_date#total_pax#max,GRP_resort_id_checkout_date#total_pax#min,GRP_resort_id_checkout_date#total_pax#median,GRP_resort_id_checkout_date#total_pax#mean,GRP_resort_id_checkout_date#roomnights#std,GRP_resort_id_checkout_date#roomnights#max,GRP_resort_id_checkout_date#roomnights#min,GRP_resort_id_checkout_date#roomnights#median,GRP_resort_id_checkout_date#roomnights#mean,GRP_resort_id_checkout_date#booking_in_advance_days#std,GRP_resort_id_checkout_date#booking_in_advance_days#max,GRP_resort_id_checkout_date#booking_in_advance_days#min,GRP_resort_id_checkout_date#booking_in_advance_days#median,GRP_resort_id_checkout_date#booking_in_advance_days#mean,GRP_resort_id_checkout_date#numberofadults#min_mean,GRP_resort_id_checkout_date#numberofchildren#min_mean,GRP_resort_id_checkout_date#total_pax#min_mean,GRP_resort_id_checkout_date#roomnights#min_mean,GRP_resort_id_checkout_date#booking_in_advance_days#min_mean
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,3,46,F,7.0,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,F,3,1,3,1,2.0,C,1,7.706428,0,1,2,14,4,2018,3,5,95,14,4,14,4,2018,95,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e0740856

In [19]:
df.shape
df.head()

(488189, 308)

,reservation_id,booking_date,checkin_date,checkout_date,memberid,numberofadults,numberofchildren,total_pax,persontravellingid,member_age_buckets,state_code_residence,resort_id,resort_region_code,resort_type_code,state_code_resort,cluster_code,channel_code,booking_type_code,room_type_booked_code,roomnights,season_holidayed_code,reservationstatusid_code,main_product_code,amount_spent_per_room_night_scaled,booking_in_advance_days,booking_roomnights,total_persons_travelling,checkin_date_week,checkin_date_month,checkin_date_year,checkin_date_dayofweek,checkin_date_dayofmonth,checkin_date_dayofyear,checkout_date_week,checkout_date_dayofweek,booking_date_week,booking_date_month,booking_date_year,booking_date_dayofyear,resort_id_checkin_date,resort_id_checkout_date,resort_id_booking_date,resort_id_channel_code,resort_id_booking_type_code,resort_id_reservationstatusid_code,resort_id_resort_type_code,resort_id_cluster_code,resort_id_room_type_booked_code,persontravellingid_member_age_buckets,persontravellingid_state_code_residence,...,GRP_resort_id_checkin_date#total_pax#std,GRP_resort_id_checkin_date#total_pax#max,GRP_resort_id_checkin_date#total_pax#min,GRP_resort_id_checkin_date#total_pax#median,GRP_resort_id_checkin_date#total_pax#mean,GRP_resort_id_checkin_date#roomnights#std,GRP_resort_id_checkin_date#roomnights#max,GRP_resort_id_checkin_date#roomnights#min,GRP_resort_id_checkin_date#roomnights#median,GRP_resort_id_checkin_date#roomnights#mean,GRP_resort_id_checkin_date#booking_in_advance_days#std,GRP_resort_id_checkin_date#booking_in_advance_days#max,GRP_resort_id_checkin_date#booking_in_advance_days#min,GRP_resort_id_checkin_date#booking_in_advance_days#median,GRP_resort_id_checkin_date#booking_in_advance_days#mean,GRP_resort_id_checkin_date#numberofadults#min_mean,GRP_resort_id_checkin_date#numberofchildren#min_mean,GRP_resort_id_checkin_date#total_pax#min_mean,GRP_resort_id_checkin_date#roomnights#min_mean,GRP_resort_id_checkin_date#booking_in_advance_days#min_mean,GRP_resort_id_checkout_date#numberofadults#std,GRP_resort_id_checkout_date#numberofadults#max,GRP_resort_id_checkout_date#numberofadults#min,GRP_resort_id_checkout_date#numberofadults#median,GRP_resort_id_checkout_date#numberofadults#mean,GRP_resort_id_checkout_date#numberofchildren#std,GRP_resort_id_checkout_date#numberofchildren#max,GRP_resort_id_checkout_date#numberofchildren#min,GRP_resort_id_checkout_date#numberofchildren#median,GRP_resort_id_checkout_date#numberofchildren#mean,GRP_resort_id_checkout_date#total_pax#std,GRP_resort_id_checkout_date#total_pax#max,GRP_resort_id_checkout_date#total_pax#min,GRP_resort_id_checkout_date#total_pax#median,GRP_resort_id_checkout_date#total_pax#mean,GRP_resort_id_checkout_date#roomnights#std,GRP_resort_id_checkout_date#roomnights#max,GRP_resort_id_checkout_date#roomnights#min,GRP_resort_id_checkout_date#roomnights#median,GRP_resort_id_checkout_date#roomnights#mean,GRP_resort_id_checkout_date#booking_in_advance_days#std,GRP_resort_id_checkout_date#booking_in_advance_days#max,GRP_resort_id_checkout_date#booking_in_advance_days#min,GRP_resort_id_checkout_date#booking_in_advance_days#median,GRP_resort_id_checkout_date#booking_in_advance_days#mean,GRP_resort_id_checkout_date#numberofadults#min_mean,GRP_resort_id_checkout_date#numberofchildren#min_mean,GRP_resort_id_checkout_date#total_pax#min_mean,GRP_resort_id_checkout_date#roomnights#min_mean,GRP_resort_id_checkout_date#booking_in_advance_days#min_mean
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,2,0,3,46,F,7.0,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,3,3,3,F,3,1,3,1,2.0,C,1,7.706428,0,1,2,14,4,2018,3,5,95,14,4,14,4,2018,95,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,4e0740856

In [20]:
# group resort date and checkin_date for number of bookings for that day

In [21]:
unique_id = 'reservation_id'

date_cols = []

target_col = 'amount_spent_per_room_night_scaled'

cat_cols = [
'memberid',
u'resort_id_memberid',
u'memberid_channel_code', u'memberid_booking_type_code',
u'memberid_reservationstatusid_code', u'memberid_resort_type_code',
u'memberid_cluster_code', u'memberid_room_type_booked_code',
u'memberid_checkin_date', u'memberid_checkout_date',
u'memberid_booking_date', u'memberid_checkin_date_month',
u'memberid_checkin_date_week', u'memberid_checkin_date_dayofweek',
u'memberid_checkout_date_week', u'memberid_checkout_date_dayofweek',
u'memberid_booking_date_week', 
u'memberid_booking_roomnights',
    
]

cat_boost_cols = [
u'resort_id_checkin_date', u'resort_id_checkout_date',
u'resort_id_booking_date', u'resort_id_channel_code',
u'resort_id_booking_type_code', u'resort_id_reservationstatusid_code',
u'resort_id_resort_type_code', u'resort_id_cluster_code',
u'resort_id_room_type_booked_code',
u'persontravellingid_member_age_buckets',
u'persontravellingid_state_code_residence',
u'persontravellingid_state_code_resort',
u'member_age_buckets_state_code_residence',
u'member_age_buckets_state_code_resort',
u'state_code_residence_state_code_resort',
u'resort_id_booking_roomnights',
'persontravellingid', 'member_age_buckets', 'state_code_residence', 
'resort_id', 'resort_region_code', 'resort_type_code', 'state_code_resort', 'cluster_code', 'channel_code',
'booking_type_code', 'room_type_booked_code', 'season_holidayed_code', 'reservationstatusid_code', 'main_product_code',
u'checkin_date', u'checkout_date', u'booking_date',
]

columns_to_drop = []


In [22]:
from ml_modules.encoding import FreqeuncyEncoding, LabelEncoding

In [23]:
fE = FreqeuncyEncoding(categorical_columns=cat_cols, return_df=True)

In [24]:
%%time
df1 = fE.fit_transform(df)

CPU times: user 55 s, sys: 42.5 s, total: 1min 37s
Wall time: 45.4 s


In [25]:
lE = LabelEncoding(categorical_columns=cat_boost_cols, return_df=True)

In [26]:
df1 = lE.fit_transform(df1)

In [27]:
df1.shape

(488189, 308)

In [28]:
df1.drop(columns=date_cols +[unique_id, target_col] + columns_to_drop, inplace=True)

In [29]:
x_train, y_train = df1[:341424].values, train[target_col].values
x_train.shape, y_train.shape

((341424, 306), (341424,))

In [30]:
# df1.columns

In [31]:
cat_indices = [ df1.columns.tolist().index(c) for c in cat_boost_cols]

In [32]:
# cat_indices

In [33]:
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

In [34]:
from ml_modules.custom_estimator import Estimator
from ml_modules.custom_fold_generator import FoldScheme

In [35]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [36]:
est = Estimator(model=CatBoostRegressor(eval_metric='RMSE', n_estimators=20000, od_type='Iter', 
                    od_wait=200, colsample_bylevel=0.7, max_depth=6, learning_rate=0.1),
              early_stopping_rounds=200, random_state=50, validation_scheme=FoldScheme.KFold,
              eval_metric='rmse', task_type='regression', scoring_metric=rmse, n_splits=5,
              categorical_features_indices=cat_indices)

In [37]:
train_preds = est.fit_transform(x_train, y_train)

0:	learn: 7.0333273	test: 7.0359151	best: 7.0359151 (0)	total: 531ms	remaining: 2h 57m 9s
100:	learn: 0.9699456	test: 0.9748779	best: 0.9748779 (100)	total: 21.8s	remaining: 1h 11m 33s
200:	learn: 0.9643610	test: 0.9719984	best: 0.9719984 (200)	total: 42.2s	remaining: 1h 9m 14s
300:	learn: 0.9606377	test: 0.9706184	best: 0.9706184 (300)	total: 1m 3s	remaining: 1h 8m 54s
400:	learn: 0.9576945	test: 0.9699270	best: 0.9699270 (400)	total: 1m 23s	remaining: 1h 8m 13s
500:	learn: 0.9548627	test: 0.9692988	best: 0.9692954 (499)	total: 1m 44s	remaining: 1h 8m
600:	learn: 0.9523749	test: 0.9688295	best: 0.9688295 (600)	total: 2m 5s	remaining: 1h 7m 21s
700:	learn: 0.9501498	test: 0.9686833	best: 0.9686486 (667)	total: 2m 25s	remaining: 1h 6m 51s
800:	learn: 0.9477923	test: 0.9683595	best: 0.9683595 (800)	total: 2m 46s	remaining: 1h 6m 27s
900:	learn: 0.9454168	test: 0.9682011	best: 0.9681895 (890)	total: 3m 7s	remaining: 1h 6m 4s
1000:	learn: 0.9432531	test: 0.9681898	best: 0.9681128 (957)	tot

In [38]:
np.mean(est.cv_scores), np.std(est.cv_scores), est.cv_scores

(0.9665809205370497,
 0.0026292888449483713,
 [0.9681128321930859,
  0.965719205745383,
  0.9623202053036531,
  0.9665218755302944,
  0.9702304839128322])

In [1]:
# joblib.dump(est, 'cat-308-k5.pkl')

In [40]:
x_test = df1[341424:].values
test_preds = est.transform(x_test)

ml_modules/custom_estimator.py:157: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict(x) for est in self.fitted_models)), axis=1)


In [41]:
pd.DataFrame(zip(test[unique_id], test_preds), columns=['reservation_id', 
        'amount_spent_per_room_night_scaled']).to_csv('cat-308-k5-test.csv', index=False)

pd.DataFrame(zip(train[unique_id], train_preds), columns=['reservation_id', 
        'amount_spent_per_room_night_scaled']).to_csv('cat-308-k5-train.csv', index=False)